# Bifurcation equations {#sec-20230102153124}

This notebook uses $\LaTeX\newcommand{\D}{\mathrm{d}}\newcommand{\E}{\mathcal{E}}\newcommand{\order}[2][1]{#2^{(#1)}}\newcommand{\reals}{\mathbb{R}}$ custom macros.

In this chapter, the bifurcation analysis of the perfect system is performed symbolically. The starting point is the symbolic expression of the energy $(u, \lambda) \mapsto \E(u, \lambda)$ that was derived in @sec-20230208081549.

To begin with, we perform the relevant initializations and retrieve the expression of the energy.

In [1]:
import itertools

from lsk import *
from sympy import *

In [2]:
VERBOSE = False

E2, E2_dot, E3 = symbols(r"\E_2 \dot{\E}_2 \E_3")
E_ij_dot, E_ijk, E_ijkl = symbols(r"\dot{E}_{ij} E_{ijk} E_{ijkl}")
E_ikl_dot = symbols(r"\dot{E}_{ikl}")
E_il_ddot = symbols(r"\ddot{E}_{il}")
E_il_dot = symbols(r"\dot{E}_{il}")
E_ijl = symbols(r"\dot{E}_{ijl}")

t, λ = symbols(r"t \lambda")
u, u_hat, w_hat = symbols(r"u \hat{u} \hat{w}")

In [3]:
v = IndexedBase("v")
w = IndexedBase("w")
ξ = IndexedBase(r"\xi")

In [4]:
i = Idx("i")
j = Idx("j")
k = Idx("k")
l = Idx("l")
_λ = Idx(r"\lambda")

In [5]:
__index_number = itertools.count()

def new_index():
    return Idx(r"i_{" + str(next(__index_number)) + "}")
    #return Symbol(r"i_{" + str(next(__index_number)) + "}")

In [6]:
u_star = create_u_star()
E = create_E().expand()

In [7]:
#| code-fold: true
display_latex_equation(r"u^\star(\lambda)", u_star)
display_latex_equation(r"\E(u, \lambda)", E)

<IPython.core.display.Math object>

<IPython.core.display.Math object>

## Outline of the strategy

In this section, we show that, besides the fundamental branch $u^\ast(\lambda)$, other (bifurcated) equilibrium branches may pass through the critical point $(u_0, \lambda_0)$. The starting point is the characterization of an equilibrium by the stationarity of the energy, which defines all equilibrium branches as implicit functions, which can be expanded with respect to some perturbation parameter.

The approach adopted here relies on the Lyapunov–Schmidt decomposition (see below). Note that other approaches are possible, see e.g. @chak2018.

For a given value of $\lambda$ (close to the critical value $\lambda_0$), we seek an equilibrium point $u$ of the
system, such that

$$
\E_{,u}(u, \lambda; \hat{u}) = 0 \quad \text{for all} \quad \hat{u} \in U.
$$ {#eq-20230102025737}

In [8]:
res = (create_E_u() * u_hat).expand()
Eq(res, 0)

Eq(-\E_2*\dddot{u}_0*\hat{u}*\lambda**3/6 - \E_2*\ddot{u}_0*\hat{u}*\lambda**2/2 - \E_2*\dot{u}_0*\hat{u}*\lambda + \E_2*\hat{u}*u + \E_3*\ddot{u}_0*\dot{u}_0*\hat{u}*\lambda**3/2 - \E_3*\ddot{u}_0*\hat{u}*\lambda**2*u/2 + \E_3*\dot{u}_0**2*\hat{u}*\lambda**2/2 - \E_3*\dot{u}_0*\hat{u}*\lambda*u + \E_3*\hat{u}*u**2/2 - \E_4*\dot{u}_0**3*\hat{u}*\lambda**3/6 + \E_4*\dot{u}_0**2*\hat{u}*\lambda**2*u/2 - \E_4*\dot{u}_0*\hat{u}*\lambda*u**2/2 + \E_4*\hat{u}*u**3/6 - \ddot{\E}_2*\dot{u}_0*\hat{u}*\lambda**3/2 + \ddot{\E}_2*\hat{u}*\lambda**2*u/2 - \ddot{u}_0*\dot{\E}_2*\hat{u}*\lambda**3/2 - \dot{\E}_2*\dot{u}_0*\hat{u}*\lambda**2 + \dot{\E}_2*\hat{u}*\lambda*u + \dot{\E}_3*\dot{u}_0**2*\hat{u}*\lambda**3/2 - \dot{\E}_3*\dot{u}_0*\hat{u}*\lambda**2*u + \dot{\E}_3*\hat{u}*\lambda*u**2/2, 0)

for all $\hat{u} \in U$.

The equilibrium state $u$ is projected onto the orthogonal subspaces $V$ and $W$.
$$
u = u^\ast(\lambda) + \xi_i \, v_i + w, \quad \text{with} \quad w \in W.
$$ {#eq-20220902174235}

It follows from the orthogonality of $V$ and $W$ that $\langle v_i, w \rangle = 0$ for all $i=1, \ldots, m$. Analysis of the bifurcated branches therefore reduces to finding $\xi_1, \ldots, \xi_m \in \reals$ and $w \in W$, such that
$$
\E_{,u}[u^\ast(\lambda) + \xi_i \, v_i + w, \lambda; \hat{u}] = 0 \quad \text{for all} \quad \hat{u} \in U.
$$ {#eq-20230107164346}

The method proceeds in three steps

1. Eq. (@eq-20230107164346) is first expressed for $\hat{u} \in W$ which leads to a definition of $w$ as an implicit function of $\xi_1, \ldots, \xi_m$ and $\lambda$ (see @sec-20230102030010),
2. Eq. (@eq-20230107164346) is then expressed for $\hat{u} \in V$ which leads to a definition of $\lambda$ as an implicit function of $\xi_1, \ldots, \xi_m$ (see @sec-20230102030125),
3. finally, a parametrization $\eta$ of $\xi_1, \ldots, \xi_m$ is introduced and the Taylor expansion of $u$ and $\lambda$ with respect to $\eta$ is derived (see @sec-20230102030224).

## Elimination of $w$ {#sec-20230102030010}

We first test Eq. (@eq-20230102025737) with $\hat{w} \in W$
$$
\E_{,u}[u^\ast(\lambda) + \xi_i \, v_i + w, \lambda; \hat{w}] = 0, \quad \text{for all} \quad \hat{w} \in W.
$$ {#eq-20220825143616}

This defines $w$ as an implicit function of $\xi_1, \ldots, \xi_m$ and $\lambda$. The following asymptotic expansion about $\xi_1 = \cdots = \xi_m = 0$ and $\lambda = \lambda_0$ is postulated
$$
\begin{aligned}
w(\xi_1, \ldots, \xi_m, \lambda) ={} & \xi_i \, w_i + \bigl( \lambda - \lambda_0 \bigr) \, w_\lambda + \tfrac{1}{2} \xi_i \, \xi_j \, w_{ij}\\
&+ \tfrac{1}{2} \bigl( \lambda - \lambda_0 \bigr)^2 w_{\lambda\lambda} + \xi_i \, \bigl( \lambda - \lambda_0 \bigr) w_{i\lambda} + \cdots,
\end{aligned}
$$ {#eq-20230107172048}
where $w_i$, $w_\lambda$, $w_{ij}$, $w_{\lambda\lambda}$ and $w_{i\lambda}$ are elements of $W$ to be defined.

In what follows, Eq. (@eq-20230107172048) is plugged into Eq. (@eq-20220825143616), which then delivers an asymptotic expansion of the stationarity equation. We will nullify the terms of order up to 2 in $\xi_i$ and $\lambda$.

It is observed that the residual $\E_{,u}$ involves $\E_3(u, u, \hat{u})$ and $\E_4(u, u, u, \hat{u})$. Since all terms of $u$ are of order 1 or more in $\xi_i$ and $\lambda$, $\E_4(u, u, u, \hat{u})$ will not be considered below. However $\E_3(u, u, \hat{u})$ will need to be properly defined index-wise. Therefore, we need to defined two equivalent expressions of $u$ with two different set of indices.

In [9]:
def _λ():
    return Symbol(r"\lambda")

def _w():
    i = new_index()
    j = new_index()
    k = new_index()
    l = new_index()
    λ_ = Idx(r"\lambda")
    return (t * ξ[i] * w[i] + _λ() * w[λ_] 
            + (t**2 * ξ[j] * ξ[k] * w[j, k]
               + 2 * _λ() * t * ξ[l] * w[l, λ_]
               + _λ() * _λ() * w[λ_, λ_]) / 2)
    
def _u():
    i = new_index()
    return (create_u_star(_λ) + t * ξ[i] * v[i] + _w())

In [10]:
res_w = expand(create_E_u(u_fun=_u) * w_hat)

We can then nullify the various coefficients of this polynomial expansion, starting with the terms of order 1 in $\xi_i$, which delivers the following variational problem

In [11]:
Eq(res_w.coeff(t, 1).subs(λ, 0), 0)

Eq(\E_2*\hat{w}*\xi[i_{0}]*v[i_{0}] + \E_2*\hat{w}*\xi[i_{1}]*w[i_{1}], 0)

for all $\hat{w} \in W$. Since $v_i \in V$, we have: $\E_2(v_i, \bullet) = 0$ and the above variational problem reduces to

$$
\E_2(w_i, \hat{w}) = 0 \quad \text{for all} \quad \hat{w} \in W.
$$

As argued in @sec-20230107173921, this leads to $w_i = 0$.

Similarly, we find the term or of order 1 in $\lambda$, and evaluate it at $\xi_1 = \cdots = \xi_m = 0$. Unfortunately, substitutions do not work with indexed quantities. Since all occurences of $\xi_i$ have been scaled by $t$ so as to keep track of the order of each term, a workaround is to set $t$ to 0. This delivers the following variational problem

In [12]:
res_w.coeff(λ, 1).subs(t, 0)

\E_2*\hat{w}*w[\lambda]

for all $\hat{w} \in W$. The same argument leads to $w_\lambda = 0$.

Similarly again, for the term of order 2 in $\lambda$

In [13]:
(eq := Eq(res_w.coeff(λ, 2).subs(t, 0), 0))

Eq(\E_2*\hat{w}*w[\lambda, \lambda]/2 + \E_3*\hat{w}*w[\lambda]**2/2 + \dot{\E}_2*\hat{w}*w[\lambda], 0)

for all $\hat{w} \in W$. Since it was found that $w_\lambda = 0$, the above problem again reduces to $
\E_2(w_{\lambda\lambda}, \hat{w}) = 0$ for all $\hat{w} \in W$ and $w_{\lambda\lambda} = 0$.

In [14]:
eq.subs(w[Idx(r"\lambda")], 0)

Eq(\E_2*\hat{w}*w[\lambda, \lambda]/2, 0)

To sum up, we have found so far that $w_i = w_\lambda = w_{\lambda\lambda} = 0$. At this point, it is interesting to override the function `_w()`, taking into account all these simplifications.

In [15]:
def _w():
    i = new_index()
    j = new_index()
    k = new_index()
    l = new_index()
    return t**2 * ξ[j] * ξ[k] * w[j, k] / 2 + _λ() * t * ξ[l] * w[l, Idx(r"\lambda")]

We can then re-evaluate the residual

In [16]:
res_w = expand(create_E_u(u_fun=_u) * w_hat)

The $\xi_i \, \xi_j$ terms deliver the following variational problem

In [17]:
lhs = res_w.coeff(t, 2).subs(λ, 0).expand()
Eq(lhs, 0)

Eq(\E_2*\hat{w}*\xi[i_{52}]*\xi[i_{53}]*w[i_{52}, i_{53}]/2 + \E_3*\hat{w}*\xi[i_{55}]*\xi[i_{60}]*v[i_{55}]*v[i_{60}]/2, 0)

for all $\hat{w} \in W$ and $\xi_1, \ldots, \xi_m \in \reals$ (small enough). This variational problem  should be understood as

$$
\xi_i \, \xi_j \, \bigl[ \E_2(w_{ij}, \hat{w}) + \E_3(v_i, v_j, \hat{w}) \bigr] = 0
\quad \text{for all} \quad \hat{w} \in W.
$$

for all $\hat{w} \in W$ and $\xi_1, \ldots, \xi_m$ *small enough*. Since the above expression is homogeneous in $\xi_i$, it must in fact vanish *for all* $\xi_1, \ldots, \xi_m$. The resulting variational problem then coincides with (@eq-20230107180410). The $\xi_i \, \xi_j$ term of the asymptotic expansion of $w$ is therefore exactly $w_{ij}$ defined in @sec-20230402152824.

Finally, the $\xi_i \, \lambda$ terms deliver the following variational problem

In [18]:
lhs = res_w.coeff(t, 1).coeff(λ, 1).subs(λ, 0).expand()
Eq(lhs, 0)

Eq(\E_2*\hat{w}*\xi[i_{54}]*w[i_{54}, \lambda] + \E_3*\dot{u}_0*\hat{w}*\xi[i_{55}]*v[i_{55}]/2 + \E_3*\dot{u}_0*\hat{w}*\xi[i_{60}]*v[i_{60}]/2 - \E_3*\dot{u}_0*\hat{w}*\xi[i_{65}]*v[i_{65}] + \dot{\E}_2*\hat{w}*\xi[i_{65}]*v[i_{65}], 0)

The above expression is written with standardized index names, delivering the following variational problem.

In [19]:
terms = lhs.args
lhs = 0

for term in terms:
    d = get_contraction_structure(term)
    (i_,), = d.keys()
    lhs += term.subs(i_, i)

Eq(lhs, 0)

Eq(\E_2*\hat{w}*\xi[i]*w[i, \lambda] + \dot{\E}_2*\hat{w}*\xi[i]*v[i], 0)

for all $\hat{w} \in W$ and $\xi_1, \ldots, \xi_m \in \reals$. We recognize the variational problem (@eq-20230107180501). In other words, the $\xi_i \, \lambda$ term of the asymptotic expansion of $w$ is $w_{i\lambda}$ defined in @sec-20230402152824.

So far, we have found the following expansion of $w$

$$
w(\xi_1, \ldots, \xi_m, \lambda) = \tfrac{1}{2} \xi_i \, \xi_j \, w_{ij} + \xi_i \, \bigl( \lambda - \lambda_0 \bigr) w_{i\lambda} + \cdots,
$$

where $w_{ij}$ and $w_{i\lambda}$ are defined by Eqs. (@eq-20230107180410) and (@eq-20230107180501), respectively.

## Elimination of $\lambda$ {#sec-20230102030125}

We now test Eq. (@eq-20230102025737) with $\hat{u} = v_i$ ($i = 1, \ldots, m$)

$$
\E_{,u}[u^\ast(\lambda) + \xi_j \, v_j + w(\xi_1, \ldots, \xi_m, \lambda), \lambda; v_i] = 0, \quad \text{for all} \quad i = 1, \ldots, m.
$$ {#eq-20230110092313}

This now defines $\lambda$ as an implicit function of $\xi_1, \ldots, \xi_m$. The following asymptotic expansion
about $\xi_1 = \cdots = \xi_m = 0$ is postulated

$$
\lambda(\xi_1, \ldots, \xi_m) = \xi_i \, \lambda_i + \tfrac{1}{2} \, \xi_i \, \xi_j \, \lambda_{ij} + \ldots
$$ {#eq-20230110091031}

where $\lambda_i$ and $\lambda_{ij}$ are scalar coefficients to be defined below.

We first implement the function `_λ()` that creates (with unique indices) the SymPy expression for the asymptotic expansions of  $(\xi_1, \ldots, \xi_m) \mapsto \lambda(\xi_1, \ldots, \xi_m)$ according to Eq. (@eq-20230110091031). Since the functions `_u()` and `_w()` call `_λ()` internally, there is no need to redefine them to implement the asymptotic expansions of $(\xi_1, \ldots, \xi_m, \lambda) \mapsto w(\xi_1, \ldots, \xi_m, \lambda)$ and $(\xi_1, \ldots, \xi_m) \mapsto u(\xi_1, \ldots, \xi_m)$ according to Eqs. (@eq-20230107172048) and (@eq-20220902174235).

Note that, although $\lambda$ is expanded to second-order in $\xi_i$, we will consider third-order terms of the energy below. Therefore, we must make sure we define the third-order term of $\lambda$. It will hopefully disappear later on.

In [20]:
λ = IndexedBase(r"\lambda")
λ_tilde = Symbol(r"\tilde{\lambda}")

def _λ():
    i = new_index()
    j = new_index()
    k = new_index()
    return t * ξ[i] * λ[i] + t**2 / 2 * ξ[j] * ξ[k] * λ[j, k] + t**3 * λ_tilde + O(t**4)

Then, the above expansions are plugged into the general expression of $\E_{,u}(u, \lambda; \bullet)$ and tested with $\hat{v} \in V$, see Eq. (@eq-20230110092313).

In [21]:
res_v = expand(create_E_u(u_fun=_u, λ_fun=_λ) * v[i]).removeO()

We get an asymptotic expansion of the residual with respect to $\xi_1, \ldots, \xi_m$. This residual must vanish for all $\xi_i$ sufficiently small. Therefore, all coefficients of the expansion `res_v` must be nullified at all orders and for all $\xi_i$ sufficiently small. Considering the expansion up to third order, we get after simplification only *two* bifurcation equations, since the terms of order 0 and 1 are identically null, as verified below.

In [22]:
assert res_v.coeff(t, 0) == 0
assert res_v.coeff(t, 1).subs(E2 * v[i], 0) == 0

We now consider the terms of order 2 in $\xi_i$

In [23]:
lhs = res_v.coeff(t, 2).expand().subs(E2 * v[i], 0)
Eq(lhs, 0)

Eq(\E_3*\dot{u}_0**2*\lambda[i_{142}]*\lambda[i_{180}]*\xi[i_{142}]*\xi[i_{180}]*v[i]/2 - \E_3*\dot{u}_0**2*\lambda[i_{217}]*\lambda[i_{221}]*\xi[i_{217}]*\xi[i_{221}]*v[i] + \E_3*\dot{u}_0**2*\lambda[i_{258}]*\lambda[i_{261}]*\xi[i_{258}]*\xi[i_{261}]*v[i]/2 + \E_3*\dot{u}_0*\lambda[i_{142}]*\xi[i_{142}]*\xi[i_{179}]*v[i]*v[i_{179}]/2 + \E_3*\dot{u}_0*\lambda[i_{180}]*\xi[i_{141}]*\xi[i_{180}]*v[i]*v[i_{141}]/2 - \E_3*\dot{u}_0*\lambda[i_{217}]*\xi[i_{217}]*\xi[i_{220}]*v[i]*v[i_{220}] + \E_3*\xi[i_{141}]*\xi[i_{179}]*v[i]*v[i_{141}]*v[i_{179}]/2 + \dot{\E}_2*\dot{u}_0*\lambda[i_{217}]*\lambda[i_{221}]*\xi[i_{217}]*\xi[i_{221}]*v[i] - \dot{\E}_2*\dot{u}_0*\lambda[i_{258}]*\lambda[i_{261}]*\xi[i_{258}]*\xi[i_{261}]*v[i] + \dot{\E}_2*\lambda[i_{217}]*\xi[i_{217}]*\xi[i_{220}]*v[i]*v[i_{220}], 0)

We again standardize the indices.

In [24]:
terms = lhs.args
lhs = 0
for term_ in terms:
    d = get_contraction_structure(term_)
    indices = next(iter(d.keys()))
    assert len(indices) == 2
    j_, k_ = indices
    term = term_.subs({j_: j, k_: k})
    lhs += term
    if VERBOSE:
        display_latex_equation(term_, term)
Eq(lhs, 0)

Eq(\E_3*\xi[j]*\xi[k]*v[i]*v[j]*v[k]/2 + \dot{\E}_2*\lambda[j]*\xi[j]*\xi[k]*v[i]*v[k], 0)

And, introducing the symbols $\dot{E}_{ij}$ and $E_{ijk}$, defined by Eqs. (@eq-20230125062505) and (@eq-20230125062510), respectively, we get the first equation.

In [25]:
lhs = lhs.subs({
    E3 * v[i] * v[j] * v[k]: E_ijk,
    λ[j] * ξ[j] * ξ[k] * E2_dot * v[i] * v[k]: λ[k] * ξ[j] * ξ[k] * E_ij_dot
})
(eq1 := Eq(lhs, 0))

Eq(E_{ijk}*\xi[j]*\xi[k]/2 + \dot{E}_{ij}*\lambda[k]*\xi[j]*\xi[k], 0)

which also reads

$$
\xi_j \, \xi_k \, \bigl( \tfrac{1}{2} E_{ijk} + \lambda_k \, \dot{E}_{ij} \bigr) = 0,
$$ {#eq-20230115101130}

for all $\xi_1, \ldots, \xi_m \in \reals$.

::: {.callout-note}
Eq. (@eq-20230115101130) results from an asymptotic expansion w.r.t. the $\xi_i$. As such it needs only be satisfied for all $\xi_1, \ldots, \xi_m$ *small enough*. However, this equation is homogeneous of degree 2 w.r.t. the $\xi_i$. Therefore, it indeed holds for all $\xi_1, \ldots, \xi_m \in \reals$.
:::

Turning now to the term of third-order in $\xi_i$, the expression with unique indices is **hudge**. Using in each term standard index names ($j$, $k$, $l$) reduces the expression considerably (since many terms cancel out).

In [26]:
lhs = res_v.coeff(t, 3)
lhs = lhs.subs(E2 * v[i], 0).expand()

terms = lhs.args
lhs = 0

for term in terms:
    d = get_contraction_structure(term)
    jkl_ = next(iter(d.keys()))
    if len(jkl_) == 3:
        j_, k_, l_ = jkl_
        lhs += term.subs({j_: Idx("j"), k_: Idx("k"), l_: Idx("l")})
    else:
        display(term)
        lhs += term
        
(eq2 := Eq(lhs, 0))

Eq(-\E_3*\dot{u}_0*\lambda[j, k]*\xi[j]*\xi[k]*\xi[l]*v[i]*v[l]/4 + \E_3*\dot{u}_0*\lambda[k, l]*\xi[j]*\xi[k]*\xi[l]*v[i]*v[j]/4 + \E_3*\lambda[j]*\xi[j]*\xi[k]*\xi[l]*v[i]*v[k]*w[l, \lambda]/2 + \E_3*\lambda[j]*\xi[j]*\xi[k]*\xi[l]*v[i]*v[l]*w[k, \lambda]/2 + \E_3*\xi[j]*\xi[k]*\xi[l]*v[i]*v[j]*w[k, l]/4 + \E_3*\xi[j]*\xi[k]*\xi[l]*v[i]*v[l]*w[j, k]/4 + \E_4*\xi[j]*\xi[k]*\xi[l]*v[i]*v[j]*v[k]*v[l]/6 + \ddot{\E}_2*\lambda[j]*\lambda[k]*\xi[j]*\xi[k]*\xi[l]*v[i]*v[l]/2 + \dot{\E}_2*\lambda[j, k]*\xi[j]*\xi[k]*\xi[l]*v[i]*v[l]/2 + \dot{\E}_2*\lambda[j]*\lambda[k]*\xi[j]*\xi[k]*\xi[l]*v[i]*w[l, \lambda] + \dot{\E}_2*\lambda[j]*\xi[j]*\xi[k]*\xi[l]*v[i]*w[k, l]/2 + \dot{\E}_3*\lambda[j]*\xi[j]*\xi[k]*\xi[l]*v[i]*v[k]*v[l]/2, 0)

And we get the second equation  (which is homogeneous of degree 3 w.r.t. the $\xi_i$ and must therefore hold even for large values of these parameters)

$$
\begin{aligned}[b]
\xi_j \, \xi_k \, \xi_l \, \bigl[
\tfrac{1}{2} \E_3(v_i, v_j, w_{kl})
+ \lambda_l \, \E_3(v_i, v_j, w_{k\lambda})
+ \tfrac{1}{6} \E_4(v_i, v_j, v_k, v_l) &\\
+ \tfrac{1}{2} \lambda_l \, \dot{\E}_2(v_i, w_{jk})
+ \lambda_k \, \lambda_l \, \dot{\E}_2(v_i, w_{j\lambda})
+ \tfrac{1}{2} \lambda_{kl} \, \dot{\E}_2(v_i, v_j) &\\
+ \tfrac{1}{2} \lambda_l \, \dot{\E}_3(v_i, v_j, v_k)
+ \tfrac{1}{2} \lambda_k \, \lambda_l \, \ddot{\E}_2(v_i, v_j)
\bigr] &= 0,
\end{aligned}
$$ {#eq-20230115101147}
for all $\xi_1, \ldots, \xi_m \in \reals$.

## Simplification of the bifurcation equations

In this section, Eqs. (@eq-20230115101130) and (@eq-20230115101147) are simplified to deliver the bifurcation equations (@eq-20230125062336) and (@eq-20230124205642).

### Parametrization of the bifurcated branch

A parametrization of the bifurcated branch is first introduced. The bifurcated branch is a curve $(u, \lambda) \in \reals ^ {m + 1}$, which is parametrized by $\eta$: $[u(\eta), \lambda(\eta)]$, with $u(0) = u_0$ and $\lambda(0) = \lambda_0$; primed quantities denoting derivatives with respect to $\eta$, we introduce

$$
\order[1]{\xi_i} = \xi_i'(0), \quad \order[2]{\xi_i} = \xi_i''(0), \quad \ldots, \quad \order[1]{\lambda} = \lambda'(0), \quad \ldots
$$

In @sec-20230102030125, $\lambda$ was defined as a function of $\xi_1, \ldots, \xi_m$. Therefore, from the chain rule

$$
\lambda'(\eta) = \frac{\partial\lambda}{\partial\xi_i} \, \xi_i'(\eta) \quad \text{and} \quad \lambda''(\eta)  = \frac{\partial^2\lambda}{\partial\xi_i \, \partial\xi_j} \, \xi_i'(\eta) \, \xi_j'(\eta) + \frac{\partial\lambda}{\partial\xi_i} \, \xi_i''(\eta)
$$

and, at $\eta = 0$

$$
\order[1]{\lambda} = \lambda_i \, \order[1]{\xi_i} \quad \text{and} \quad \order[2]{\lambda} = \lambda_{ij} \, \order[1]{\xi_i} \, \order[1]{\xi_j} + \lambda_i \, \order[2]{\xi_i}.
$$ {#eq-20230116205753}

In [27]:
ξ1 = IndexedBase(r"{\order[1]{\xi}}")
ξ2 = IndexedBase(r"{\order[2]{\xi}}")
λ1, λ2 = symbols(r"{\order[1]{\lambda}} {\order[2]{\lambda}}")

### Simplification of the first bifurcation equation

We first observe that, for fixed $i = 1, \ldots, m$, Eq. (@eq-20230115101130) is of the form $L_{jk} \, \xi_j \, \xi_k = 0$ for all $\xi_1, \ldots, \xi_m$. Derivation with respect to $\xi_l$ leads to

$$
0 = L_{jk} \, \delta_{jl} \, \xi_k + L_{jk} \, \xi_j \, \delta_{kl} = L_{lk} \, \xi_k + L_{jl} \, \xi_j = \bigl(L_{kl} +L_{lk} \bigr) \, \xi_l,
$$

for all $\xi_1, \ldots, \xi_m \in \reals$. The above identity leads to: $L_{kl} + L_{lk} = 0$. Application to
Eq. (@eq-20230115101130) delivers the following identity, to be used below

$$
E_{ijk} + \lambda_k \, \dot{E}_{ij} + \lambda_j \, \dot{E}_{ik} = 0.
$$ {#eq-20230120210008}

Eq. (@eq-20230115101130) holds for all $\xi_1, \ldots, \xi_m$. In particular, it holds for $\xi_i = \order[1]{\xi_i}$ and using
Eq. (@eq-20230116205753) delivers

In [28]:
eq1.subs(ξ, ξ1).subs(λ[k] * ξ1[k], λ1)

Eq(E_{ijk}*{\order[1]{\xi}}[j]*{\order[1]{\xi}}[k]/2 + \dot{E}_{ij}*{\order[1]{\lambda}}*{\order[1]{\xi}}[j], 0)

which coincides with the **first bifurcation equation** (@eq-20230125062336).

### Simplification of the second bifurcation equation

We first isolate the LHS of this equation

In [29]:
(lhs2 := eq2.lhs)

-\E_3*\dot{u}_0*\lambda[j, k]*\xi[j]*\xi[k]*\xi[l]*v[i]*v[l]/4 + \E_3*\dot{u}_0*\lambda[k, l]*\xi[j]*\xi[k]*\xi[l]*v[i]*v[j]/4 + \E_3*\lambda[j]*\xi[j]*\xi[k]*\xi[l]*v[i]*v[k]*w[l, \lambda]/2 + \E_3*\lambda[j]*\xi[j]*\xi[k]*\xi[l]*v[i]*v[l]*w[k, \lambda]/2 + \E_3*\xi[j]*\xi[k]*\xi[l]*v[i]*v[j]*w[k, l]/4 + \E_3*\xi[j]*\xi[k]*\xi[l]*v[i]*v[l]*w[j, k]/4 + \E_4*\xi[j]*\xi[k]*\xi[l]*v[i]*v[j]*v[k]*v[l]/6 + \ddot{\E}_2*\lambda[j]*\lambda[k]*\xi[j]*\xi[k]*\xi[l]*v[i]*v[l]/2 + \dot{\E}_2*\lambda[j, k]*\xi[j]*\xi[k]*\xi[l]*v[i]*v[l]/2 + \dot{\E}_2*\lambda[j]*\lambda[k]*\xi[j]*\xi[k]*\xi[l]*v[i]*w[l, \lambda] + \dot{\E}_2*\lambda[j]*\xi[j]*\xi[k]*\xi[l]*v[i]*w[k, l]/2 + \dot{\E}_3*\lambda[j]*\xi[j]*\xi[k]*\xi[l]*v[i]*v[k]*v[l]/2

We again plug Eq. (@eq-20230116205753) into (@eq-20230115101147).

In [30]:
(lhs2a := lhs2.subs(ξ, ξ1).subs({
    λ[j] * ξ1[j]: λ1,
    λ[k] * ξ1[k]: λ1
}).expand())

-\E_3*\dot{u}_0*\lambda[j, k]*v[i]*v[l]*{\order[1]{\xi}}[j]*{\order[1]{\xi}}[k]*{\order[1]{\xi}}[l]/4 + \E_3*\dot{u}_0*\lambda[k, l]*v[i]*v[j]*{\order[1]{\xi}}[j]*{\order[1]{\xi}}[k]*{\order[1]{\xi}}[l]/4 + \E_3*{\order[1]{\lambda}}*v[i]*v[k]*w[l, \lambda]*{\order[1]{\xi}}[k]*{\order[1]{\xi}}[l]/2 + \E_3*{\order[1]{\lambda}}*v[i]*v[l]*w[k, \lambda]*{\order[1]{\xi}}[k]*{\order[1]{\xi}}[l]/2 + \E_3*v[i]*v[j]*w[k, l]*{\order[1]{\xi}}[j]*{\order[1]{\xi}}[k]*{\order[1]{\xi}}[l]/4 + \E_3*v[i]*v[l]*w[j, k]*{\order[1]{\xi}}[j]*{\order[1]{\xi}}[k]*{\order[1]{\xi}}[l]/4 + \E_4*v[i]*v[j]*v[k]*v[l]*{\order[1]{\xi}}[j]*{\order[1]{\xi}}[k]*{\order[1]{\xi}}[l]/6 + \ddot{\E}_2*{\order[1]{\lambda}}**2*v[i]*v[l]*{\order[1]{\xi}}[l]/2 + \dot{\E}_2*{\order[1]{\lambda}}**2*v[i]*w[l, \lambda]*{\order[1]{\xi}}[l] + \dot{\E}_2*{\order[1]{\lambda}}*v[i]*w[k, l]*{\order[1]{\xi}}[k]*{\order[1]{\xi}}[l]/2 + \dot{\E}_2*\lambda[j, k]*v[i]*v[l]*{\order[1]{\xi}}[j]*{\order[1]{\xi}}[k]*{\order[1]{\xi}}[l]/2 + \dot{\E}

The term in $\order[1]{\xi}_j \, \order[1]{\xi}_k \, \order[1]{\xi}_l$ reads

In [31]:
cofactor3 = ξ1[j] * ξ1[k] * ξ1[l]
(term3 := lhs2a.coeff(ξ1[j]).coeff(ξ1[k]).coeff(ξ1[l]))

-\E_3*\dot{u}_0*\lambda[j, k]*v[i]*v[l]/4 + \E_3*\dot{u}_0*\lambda[k, l]*v[i]*v[j]/4 + \E_3*v[i]*v[j]*w[k, l]/4 + \E_3*v[i]*v[l]*w[j, k]/4 + \E_4*v[i]*v[j]*v[k]*v[l]/6 + \dot{\E}_2*\lambda[j, k]*v[i]*v[l]/2

Owing to the contraction with $\order[1]{\xi}_j$, $\order[1]{\xi}_k$ and $\order[1]{\xi}_l$, the indices $j$, $k$ and $l$ can be permuted in some terms.

In [32]:
(term3a := term3.subs({
    λ[k, l] * E3 * v[j]: λ[j, k] * E3 * v[l],
    v[l] * w[j, k]: v[j] * w[k, l]
}))

\E_3*v[i]*v[j]*w[k, l]/2 + \E_4*v[i]*v[j]*v[k]*v[l]/6 + \dot{\E}_2*\lambda[j, k]*v[i]*v[l]/2

Similarly, the following symmetrization can be performed

$$
\order[1]{\xi}_j \, \order[1]{\xi}_k \, \order[1]{\xi}_l \, \bigl[ \E_4(v_i, v_j, v_k, v_l) + 3\E_3(v_i, v_j, w_{kl}) \bigr]
= E_{ijkl} \, \order[1]{\xi}_j \, \order[1]{\xi}_k \, \order[1]{\xi}_l,
$$

where $E_{ijkl}$ is defined by Eq. (@eq-20230124210049).

In [33]:
(term3b := term3a.subs(E4 * v[i] * v[j] * v[k] * v[l], E_ijkl - 3 * E3 * v[i] * v[j] * w[k, l]))

E_{ijkl}/6 + \dot{\E}_2*\lambda[j, k]*v[i]*v[l]/2

And the result is plugged back into the LHS of the bifurcation equation.

In [34]:
(lhs2b := expand(lhs2a + cofactor3 * (term3b - term3)))

E_{ijkl}*{\order[1]{\xi}}[j]*{\order[1]{\xi}}[k]*{\order[1]{\xi}}[l]/6 + \E_3*{\order[1]{\lambda}}*v[i]*v[k]*w[l, \lambda]*{\order[1]{\xi}}[k]*{\order[1]{\xi}}[l]/2 + \E_3*{\order[1]{\lambda}}*v[i]*v[l]*w[k, \lambda]*{\order[1]{\xi}}[k]*{\order[1]{\xi}}[l]/2 + \ddot{\E}_2*{\order[1]{\lambda}}**2*v[i]*v[l]*{\order[1]{\xi}}[l]/2 + \dot{\E}_2*{\order[1]{\lambda}}**2*v[i]*w[l, \lambda]*{\order[1]{\xi}}[l] + \dot{\E}_2*{\order[1]{\lambda}}*v[i]*w[k, l]*{\order[1]{\xi}}[k]*{\order[1]{\xi}}[l]/2 + \dot{\E}_2*\lambda[j, k]*v[i]*v[l]*{\order[1]{\xi}}[j]*{\order[1]{\xi}}[k]*{\order[1]{\xi}}[l]/2 + \dot{\E}_3*{\order[1]{\lambda}}*v[i]*v[k]*v[l]*{\order[1]{\xi}}[k]*{\order[1]{\xi}}[l]/2

We now consider the $\order[1]{\xi}_k \, \order[1]{\xi}_l$ term

In [35]:
cofactor2 = ξ1[k] * ξ1[l]
(term2 := lhs2b.coeff(ξ1[j], 0).coeff(ξ1[k]).coeff(ξ1[l]))

\E_3*{\order[1]{\lambda}}*v[i]*v[k]*w[l, \lambda]/2 + \E_3*{\order[1]{\lambda}}*v[i]*v[l]*w[k, \lambda]/2 + \dot{\E}_2*{\order[1]{\lambda}}*v[i]*w[k, l]/2 + \dot{\E}_3*{\order[1]{\lambda}}*v[i]*v[k]*v[l]/2

The variational problems (@eq-20230107180410) and (@eq-20230107180501) is used twice to transform $\E_3(v_i, v_j, w_{k\lambda})$

$$
\E3(v_i, v_j, w_{k\lambda}) = -\E_2(w_{ij}, w_{k\lambda}) = \dot{\E}_2(w_{ij}, v_k)
$$

In [37]:
_λ = Idx(r"\lambda")
(term2a := term2.subs({
    E3 * v[i] * v[k] * w[l, _λ]: E2_dot * w[i, k] * v[l],
    E3 * v[i] * v[l] * w[k, _λ]: E2_dot * w[i, l] * v[k]
}))

\dot{\E}_2*{\order[1]{\lambda}}*v[i]*w[k, l]/2 + \dot{\E}_2*{\order[1]{\lambda}}*v[k]*w[i, l]/2 + \dot{\E}_2*{\order[1]{\lambda}}*v[l]*w[i, k]/2 + \dot{\E}_3*{\order[1]{\lambda}}*v[i]*v[k]*v[l]/2

Finally, we introduce the definition (@eq-20230124210649) of $\dot{E}_{ijk}$

In [38]:
(term2b := term2a.subs(E3_dot * v[i] * v[k] * v[l],
                       E_ikl_dot - E2_dot * v[i] * w[k, l] - E2_dot * v[k] * w[i, l] -E2_dot * v[l] * w[i, k]).expand())

\dot{E}_{ikl}*{\order[1]{\lambda}}/2

And the result is plugged back into the LHS of the bifurcation equation.

In [39]:
(lhs2c := expand(lhs2b + cofactor2 * (term2b - term2)))

E_{ijkl}*{\order[1]{\xi}}[j]*{\order[1]{\xi}}[k]*{\order[1]{\xi}}[l]/6 + \ddot{\E}_2*{\order[1]{\lambda}}**2*v[i]*v[l]*{\order[1]{\xi}}[l]/2 + \dot{E}_{ikl}*{\order[1]{\lambda}}*{\order[1]{\xi}}[k]*{\order[1]{\xi}}[l]/2 + \dot{\E}_2*{\order[1]{\lambda}}**2*v[i]*w[l, \lambda]*{\order[1]{\xi}}[l] + \dot{\E}_2*\lambda[j, k]*v[i]*v[l]*{\order[1]{\xi}}[j]*{\order[1]{\xi}}[k]*{\order[1]{\xi}}[l]/2

We now consider the term in $\bigl[ \order[1]{\lambda} \bigr]^2 \, \order[1]{\xi}_l$

In [40]:
cofactor1 = λ1**2 * ξ1[l]
(term1 := lhs2c.coeff(ξ1[j], 0).coeff(ξ1[k], 0).coeff(ξ1[l]).coeff(λ1, 2))

\ddot{\E}_2*v[i]*v[l]/2 + \dot{\E}_2*v[i]*w[l, \lambda]

This term is simplified by testing Eq. (@eq-20230107180501), with $\hat{w} = w_{l\lambda}$ and $\hat{w} = w_{i\lambda}$, successively

$$
2\dot{\E}_2(v_i, w_{l\lambda})
= \dot{\E}_2(v_i, w_{l\lambda}) -\E_2(w_{i\lambda}, w_{l\lambda})
= \dot{\E}_2(v_i, w_{l\lambda}) + \E_2(v_l, w_{i\lambda})
= \ddot{E}_{il} - \ddot{E}_2(v_i, v_l),
$$

where $\ddot{E}_{ij}$ is defined by Eq. (@eq-20230124211207). 

In [41]:
term1a = E_il_ddot / 2

In [42]:
(lhs2d := expand(lhs2c + cofactor1 * (term1a - term1)))

E_{ijkl}*{\order[1]{\xi}}[j]*{\order[1]{\xi}}[k]*{\order[1]{\xi}}[l]/6 + \ddot{E}_{il}*{\order[1]{\lambda}}**2*{\order[1]{\xi}}[l]/2 + \dot{E}_{ikl}*{\order[1]{\lambda}}*{\order[1]{\xi}}[k]*{\order[1]{\xi}}[l]/2 + \dot{\E}_2*\lambda[j, k]*v[i]*v[l]*{\order[1]{\xi}}[j]*{\order[1]{\xi}}[k]*{\order[1]{\xi}}[l]/2

The definition (@eq-20230125062505) of $\dot{E}_{il}$ is then substituted

In [43]:
(lhs2e := lhs2d.subs(E2_dot * v[i] * v[l], E_il_dot))

E_{ijkl}*{\order[1]{\xi}}[j]*{\order[1]{\xi}}[k]*{\order[1]{\xi}}[l]/6 + \ddot{E}_{il}*{\order[1]{\lambda}}**2*{\order[1]{\xi}}[l]/2 + \dot{E}_{ikl}*{\order[1]{\lambda}}*{\order[1]{\xi}}[k]*{\order[1]{\xi}}[l]/2 + \dot{E}_{il}*\lambda[j, k]*{\order[1]{\xi}}[j]*{\order[1]{\xi}}[k]*{\order[1]{\xi}}[l]/2

Upon susbtitution of $\order[1]{\xi}_j \, \order[1]{\xi}_k \, \lambda_{jk}$ with $\order[2]{\lambda} - \lambda_k \, \order[2]{\xi_k}$ \[see Eq. (@eq-20230116205753)\]

In [44]:
(lhs2f := lhs2e.subs(ξ1[j] * ξ1[k] * λ[j, k], λ2 - λ[j] * ξ2[j]).expand())

E_{ijkl}*{\order[1]{\xi}}[j]*{\order[1]{\xi}}[k]*{\order[1]{\xi}}[l]/6 + \ddot{E}_{il}*{\order[1]{\lambda}}**2*{\order[1]{\xi}}[l]/2 + \dot{E}_{ikl}*{\order[1]{\lambda}}*{\order[1]{\xi}}[k]*{\order[1]{\xi}}[l]/2 + \dot{E}_{il}*{\order[2]{\lambda}}*{\order[1]{\xi}}[l]/2 - \dot{E}_{il}*\lambda[j]*{\order[1]{\xi}}[l]*{\order[2]{\xi}}[j]/2

Finally, we use Eq. (@eq-20230120210008)

In [47]:
(lhs2g := lhs2f.subs(λ[j] * E_il_dot, -E_ijl - λ[l] * E_ij_dot).expand())

E_{ijkl}*{\order[1]{\xi}}[j]*{\order[1]{\xi}}[k]*{\order[1]{\xi}}[l]/6 + \ddot{E}_{il}*{\order[1]{\lambda}}**2*{\order[1]{\xi}}[l]/2 + \dot{E}_{ijl}*{\order[1]{\xi}}[l]*{\order[2]{\xi}}[j]/2 + \dot{E}_{ij}*\lambda[l]*{\order[1]{\xi}}[l]*{\order[2]{\xi}}[j]/2 + \dot{E}_{ikl}*{\order[1]{\lambda}}*{\order[1]{\xi}}[k]*{\order[1]{\xi}}[l]/2 + \dot{E}_{il}*{\order[2]{\lambda}}*{\order[1]{\xi}}[l]/2

Plugging again Eq. (@eq-20230116205753)

In [50]:
(lhs2h := lhs2g.subs(λ[l] * ξ1[l], λ1).expand())

E_{ijkl}*{\order[1]{\xi}}[j]*{\order[1]{\xi}}[k]*{\order[1]{\xi}}[l]/6 + \ddot{E}_{il}*{\order[1]{\lambda}}**2*{\order[1]{\xi}}[l]/2 + \dot{E}_{ijl}*{\order[1]{\xi}}[l]*{\order[2]{\xi}}[j]/2 + \dot{E}_{ij}*{\order[1]{\lambda}}*{\order[2]{\xi}}[j]/2 + \dot{E}_{ikl}*{\order[1]{\lambda}}*{\order[1]{\xi}}[k]*{\order[1]{\xi}}[l]/2 + \dot{E}_{il}*{\order[2]{\lambda}}*{\order[1]{\xi}}[l]/2

The second bifurcation equation finally reads

In [54]:
#| code-fold: true
display_latex_equation(expand(2 * lhs2h), 0)

<IPython.core.display.Math object>

and Eq. (@eq-20230124205642) is retrieved.

## Taylor expansion of the bifurcated branch

We now turn to $w$, which was defined in @sec-20230102030010 as an implicit function of $\xi_1, \ldots, \xi_m$ and
$\lambda$. It is now defined as a function of the arc parameter $\eta$ as follows: $w(n) = w[\xi_1(\eta), \ldots, \xi_m(\eta), \lambda(\eta)]$.
From the chain rule
$$
w'(\eta) = \frac{\partial w}{\partial \xi_i} \, \xi_i' + \frac{\partial w}{\partial \lambda} \, \lambda'
$$
and
$$
w''(\eta) = \frac{\partial^2 w}{\partial \xi_i \, \partial \xi_j} \, \xi_i' \, \xi_j'
+ 2\frac{\partial w}{\partial \xi_i \, \partial \lambda} \, \xi_i' \, \lambda'
+ \frac{\partial w}{\partial \xi_i} \, \xi_i''
+ \frac{\partial^2 w}{\partial \lambda^2} \, \lambda^{'2}
+ \frac{\partial w}{\partial \lambda} \, \lambda''.
$$

At $\eta = 0$, the above identities reduce to
$$
w'(0) = 0
\quad \text{and} \quad
w''(0) = \order[1]{\xi_i} \, \order[1]{\xi_j} \, w_{ij}  + 2 \order[1]{\lambda} \, \order[1]{\xi_i} \, w_{i\lambda}
$$

We deduce from the above the Taylor expansion (@eq-20230528174310) of the bifurcated branch as $\eta \to 0$.